In [ ]:
from collections import defaultdict
from PIL import Image
import random
from tqdm import trange

In [ ]:
height = 2400
width = 360

In [ ]:
class Circle():
    def __init__(self, x, y, r, d,color):
        self.x = x
        self.y = y
        self.r = r
        self.d = d
        self.color = color
    def draw(self, canvas, depth_map):
        for i in range(-self.r, self.r + 1):  # Adjusted to cover all quadrants.
            for j in range(-self.r, self.r + 1):
                if i**2 + j**2 <= self.r**2:
                    x = self.x + i
                    y = self.y + j
                    if 0 <= x < width and 0 <= y < height:
                        canvas.putpixel((x, y), self.color)
                        depth_map.putpixel((x, y), self.d)


In [ ]:
def random_position():
    import random
    return (random.randint(0, width), random.randint(0, height))

In [ ]:
def random_color():
    import random
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

def random_sun_color():
    import random
    return random.choice([
        (247,192,76),
        (28,79,142),
        (174,126,126),
        (174,174,174)

    ])

In [ ]:
def get_trans_colors():
    colors = [
        (91,206,250),
        (245,169,184),
        (255,255,255)
    ]
    return random.choice(colors)


In [164]:
def doesnt_overlap(pos,circle_radius):
    for layer in entities:
        for b in entities[layer]:
            #circle
            if isinstance(b, Circle):
                # if pos in circle 
                if (pos[0] - b.x)**2 + (pos[1] - b.y)**2 <= (circle_radius)**2:
                

                    return False
        
    return True

In [165]:
raise Exception("Stop")

Exception: Stop

# stars

In [ ]:
depth_map = Image.new('L', (width, height))
canvas = Image.new('RGB', (width, height))

min_box_width = 1
min_box_height = 1

max_box_width = 4
max_box_height =4

box_layer_count = 10
box_layers=60
max_radius = 3
entities = defaultdict(list)
for layer_id in range(box_layers,0,-1):
    for box_id in trange(box_layer_count):
        #scaling based on layer
        circle_radius = int(10 + (layer_id / box_layers) * max_radius)
        layer_depth = int(255 * (layer_id / box_layers))

        max_tries = 2
        while True:
            max_tries -= 1
            if max_tries == 0:
                break
            position = random_position()
            if doesnt_overlap(position,circle_radius):
                entities[layer_id].append(Circle(*position, circle_radius, layer_depth))
                break
        
for layer_id, layer in entities.items():
    print(layer_id, len(layer))
    for b in layer:
        b.draw(canvas, depth_map)

canvas.save('canvas.jpg')
depth_map.save('canvas_depth.jpg')

In [ ]:

#fill center oval with black

center_x = width // 2
center_y = height // 2
center_r = 625
center_d = 157
center = Circle(center_x, center_y, center_r, center_d)
center.color = (0,0,0)
center.draw(canvas, depth_map)
max_levels=35


for level in trange(max_levels):
    center_x = width // 2
    center_y = height // 2
    center_r = 600-(level*50)
    center_d = max([(255-(15*max_levels)),150])+(level*3)
    center = Circle(center_x, center_y, center_r, center_d)
    center.color = (level*5,level*5,level*5)
    center.draw(canvas, depth_map)

In [ ]:

text = [
    "Trans Rights",
    "Are Human Rights",
]
text_depth = 250
#render text in side 27 font, centered horizontally and vertically on the canvas and depth map

font = ImageFont.truetype("/usr/share/fonts/truetype/ubuntu/UbuntuMono-B.ttf", 121)
font_big = ImageFont.truetype("/usr/share/fonts/truetype/ubuntu/UbuntuMono-B.ttf", 122)
draw = ImageDraw.Draw(canvas)
depth_draw = ImageDraw.Draw(depth_map)

second_line_offset = 120
text_height = 120
for i, line in enumerate(text):
    text_width = draw.textlength(line, font)
    draw.text((int((width - text_width) // 2), int((height - text_height ) // 2 + i * second_line_offset)), line, font=font, fill=(255,00,0))
    depth_draw.text((int((width - text_width) // 2), int((height - text_height) // 2 + i * second_line_offset)), line, font=font, fill=text_depth)

#write cavas.jpg and canvas_depth.jpg
canvas.save('canvas.jpg')
depth_map.save('canvas_depth.jpg')

# Image In Stars

In [ ]:
class CanvasImage():
    def __init__(self,file_path, x,y,depth):
        self.file_path = file_path
        self.image = Image.open(file_path)
        # transparency
        self.image = self.image.convert("RGBA")
        self.width = self.image.width
        self.height = self.image.height
        self.pixels = self.image.load()
        self.x = x
        self.y = y
        self.depth = depth
    def draw(self, canvas, depth_map):
        for ix in range(self.width):
            x = self.x + ix
            for iy in range(self.height):
                y = self.y + iy
                if 0 <= x < width and 0 <= y < height:
                    #with transparency
                    r,g,b,a = self.pixels[ix,iy]
                    if a > 0:
                        canvas.putpixel((x, y), (r,g,b))
                        depth_map.putpixel((x, y), self.depth)

In [ ]:
class BubbleCanvasImage():
    def __init__(self,file_path, x,y,depth,max_image_depth):
        self.file_path = file_path
        self.image = Image.open(file_path)
        # transparency
        self.image = self.image.convert("RGBA")
        self.width = self.image.width
        self.height = self.image.height
        self.pixels = self.image.load()
        self.x = x
        self.y = y
        self.depth = depth
        self.max_image_depth = max_image_depth
        self.image_center = (self.width // 2, self.height // 2)

    def get_falloff_distance(self, x, y,rate):
        dx = abs(x - self.x - self.image_center[0])
        dy = abs(y - self.y - self.image_center[1])
        return (dx ** 2 + dy ** 2) ** 0.5 * rate

    def adjust_depth(self, x, y, depth):
        #start at depth and get lighter the closer to the center
        falloff_distance = self.get_falloff_distance(x, y,rate=0.45)
        falloff = falloff_distance / self.max_image_depth
        falloff = min(1, falloff)
        falloff = max(0, falloff)
        return depth - int((255 - depth) * falloff)

    def draw(self, canvas, depth_map):
        for ix in range(self.width):
            x = self.x + ix
            for iy in range(self.height):
                y = self.y + iy
                if 0 <= x < width and 0 <= y < height:
                    #with transparency
                    r,g,b,a = self.pixels[ix,iy]
                    if a > 0:
                        canvas.putpixel((x, y), (r,g,b))
                        depth_map.putpixel((x, y), self.adjust_depth(x,y,self.depth))

In [170]:
class ColorExtractorAndGenerator():
    def __init__(self, image_obj, max_colors=15, avg_color_amount= 15,ignore_most_common=1):
        self.image_obj = image_obj
        self.colors = self.extract_colors(avg_color_amount, max_colors,ignore_most_common)
        print(self.colors)
    def group_color(self, color, color_counts, avg_color_grouping_amount):
        for cc_color in color_counts.keys():
            if abs(color[0] - cc_color[0]) < avg_color_grouping_amount and \
               abs(color[1] - cc_color[1]) < avg_color_grouping_amount and \
               abs(color[2] - cc_color[2]) < avg_color_grouping_amount:
                return cc_color
        return color
    def extract_colors(self, avg_color_grouping_amount, max_colors,ignore_most_common=1):
        extracted_colors = []
        sampling_amount = 1000
        for i in range(sampling_amount):
            x = random.randint(0, self.image_obj.width - 1)
            y = random.randint(0, self.image_obj.height - 1)
            r,g,b,a = self.image_obj.getpixel((x, y))
            extracted_colors.append((r,g,b))
        

        #sort by most common and deduplicate, reduce to max_colors based on avg_color_amount
        color_counts = defaultdict(int)
        for color in extracted_colors:
            color = self.group_color(color,color_counts,avg_color_grouping_amount)
            color_counts[color] += 1
        color_counts = sorted(color_counts.items(), key=lambda x: x[1], reverse=True)
        
        #ignore
        color_counts = color_counts[ignore_most_common:]
        color_counts = color_counts[:max_colors]

        return [color for color, count in color_counts]

    def generate(self):
        return random.choice(self.colors)


In [173]:
for file in os.listdir('input'):
    input_image = Image.open(os.path.join('input', file))
    colors = ColorExtractorAndGenerator(input_image)
    border_vertical = 100
    border_horizontal =75
    height = input_image.size[1] + 2 * border_vertical
    width = input_image.size[0] + 2 * border_horizontal

    depth_map = Image.new('L', (width, height))
    canvas = Image.new('RGB', (width, height),(175,175,175)) 
    #gradient dark to light near ( 175,175,175)

    for y in range(height):
        for x in range(width):
            d = int(175 * (1 - (abs(y - height // 2) / (height // 2))))
            canvas.putpixel((x, y), (d,d,d))

    entities = defaultdict(list)

    #background circles
    background_circles = 500
    background_circles_start_depth = 5
    background_circles_end_depth= 150

    for i in trange(background_circles):
        circle_radius = random.randint(1, 5)
        color = colors.generate()
        layer_depth = random.randint(background_circles_start_depth, background_circles_end_depth)
        max_tries = 15
        while True:
            max_tries -= 1
            if max_tries == 0:
                print("max tries")
                break
            position = random_position()
            if doesnt_overlap(position,circle_radius):
                entities[layer_depth].append(Circle(*position, circle_radius, layer_depth,color))
                break

    image_depth = 200
    max_image_depth = 110
    center_x = width // 2
    center_y = height // 2
    actual_image_x = center_x - input_image.width // 2
    actual_image_y = center_y - input_image.height // 2
    entities[image_depth].append(BubbleCanvasImage(os.path.join('input', file), actual_image_x, actual_image_y, image_depth,max_image_depth))



            
    for layer_id, layer in entities.items():
        for b in layer:
            b.draw(canvas, depth_map)


    canvas.save(os.path.join('output', file))
    depth_map.save(os.path.join('output', file.replace('.png', '_depth.png')))

[(239, 242, 244), (246, 126, 30), (15, 17, 24), (249, 54, 9), (31, 6, 38), (252, 104, 22), (251, 78, 21), (53, 34, 29), (23, 37, 31), (239, 51, 24), (43, 41, 50), (32, 15, 23), (73, 23, 0), (248, 71, 46), (218, 210, 214)]


100%|██████████| 500/500 [00:00<00:00, 4720.78it/s]
